# Browsing Books - A Recomendation Engine for Books
For CMPT3520 Machine Learning II <br/>
Annabell Rodriguez, Laura Brin, Sandra Alex

## Introduction

### Business Problem

### Evaluation Metrics

## Browsing Books

### Loading data

In [ ]:
#Loading Libraries
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# Install Altair and activate its colab renderer.
print("Installing Altair...")
!pip install git+git://github.com/altair-viz/altair.git
import altair as alt
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')
print("Done installing Altair.")

In [ ]:
#Loading Dataset
from ast import literal_eval
 
books_small = pd.read_csv('https://raw.githubusercontent.com/malcolmosh/goodbooks-10k/master/books_enriched.csv', index_col=[0],dtype={"isbn":object}, converters={"genres": literal_eval})
#note books_enriched.csv is a modified version of books.csv from goodbooks-10k dataset
users = pd.read_csv("Datasets\\Users.csv")
ratings= pd.read_csv("Datasets\\Ratings.csv")
books_large = pd.read_csv("Datasets\\Books.csv",dtype={"ISBN":object},low_memory=False)
#books3 = pd.read_csv("Datasets\\books_enriched.csv",dtype={"isbn":object})
#gr_books=pd.read_csv("Datasets\\Goodreads_books_with_genres.csv",dtype={"isbn":object})


### Books

In [ ]:
books_small.head()

In [ ]:
books_small.columns

In [ ]:
books_small.drop(["book_id","best_book_id","description","goodreads_book_id","ratings_1","ratings_2","ratings_3","ratings_4","ratings_5","ratings_count","work_id","work_ratings_count","work_text_reviews_count","authors_2"],axis=1,inplace=True, errors='ignore')

In [ ]:
books.dtypes

In [ ]:
#Cleaning code adapted from Zygmunt Zajac (github:zyggmuntz) and Olivier Simard-Handley (github:malcolmosh) -- source for updated books dataset
goodread_genres = {"art", "biography", "business", "chick-lit", "children's", "christian", "classics",
          "comics", "contemporary", "cookbooks", "crime", "books", "fantasy", "fiction",
          "gay-and-lesbian", "graphic-novels", "historical-fiction", "history", "horror",
          "humor-and-comedy", "manga", "memoir", "music", "mystery", "nonfiction", "paranormal",
          "philosophy", "poetry", "psychology", "religion", "romance", "science", "science-fiction", 
          "self-help", "suspense", "spirituality", "sports", "thriller", "travel", "young-adult"}

books_small['genres'] = books_small['genres'].apply(lambda x: [genre for genre in x if genre in goodread_genres])
     

In [ ]:
books_small["isbn"].isin(ratings["ISBN"]).value_counts()

In [ ]:
books_small["isbn"].isin(books_large["ISBN"]).value_counts()

In [ ]:
books_large.head()

In [ ]:
books_small["original_title"].isin(books_large["Book-Title"]).value_counts()

In [ ]:
books_small["isbn"].isin(books3["isbn"]).value_counts()

In [ ]:
books_large["Book-Title"].isin(books_small["original_title"]).value_counts()

Combining datasets

In [ ]:
all_books=pd.merge(books_small,books_large,how="inner",left_on="original_title",right_on="Book-Title")
all_books=all_books[all_books['language_code'] == 'eng']
all_books.head()

In [ ]:
all_books.drop(['original_title','index','title','isbn13','books_count','publishDate','isbn','language_code','Book-Author','Year-Of-Publication','Publisher','image_url','small_image_url','Image-URL-S','Image-URL-M','Image-URL-L'],axis=1,inplace=True, errors='ignore')

In [ ]:
all_books["ISBN"].isin(ratings["ISBN"]).value_counts()

In [ ]:
# Run Only Once!
all_books["average_rating"]=all_books["average_rating"].apply(lambda x: x*2)

In [ ]:
all_books.head()

### Users

In [ ]:
users.head()

In [ ]:
users['city'],users['state'],users['country']=users["Location"].str.split(",",2).str
users.drop(['Location'],axis=1,inplace=True, errors='ignore')
users.head()

In [ ]:
users["country"].fillna("Unknown", inplace = True)
users["state"].fillna("Unknown", inplace = True)
users["city"].fillna("Unknown", inplace = True)

In [ ]:
list=users["country"].value_counts().where(users["country"].value_counts()>7500,other="Other")
print(list)


In [ ]:
users["new_country"]=users["country"].apply(lambda x: x if list[x]!="Other" else "Other")
users["new_country"].value_counts()


In [ ]:
state_list=users["state"].value_counts().where(users["state"].value_counts()>5000,other="Other")
users["new_state"]=users["state"].apply(lambda x: x if state_list[x]!="Other" else "Other")
users["new_state"].value_counts()

In [ ]:
city_list=users["city"].value_counts().where(users["city"].value_counts()>1500,other="Other")
users["new_city"]=users["city"].apply(lambda x: x if city_list[x]!="Other" else "Other")
users["new_city"].value_counts()

In [ ]:
users.describe()

In [ ]:
import random
random.seed(42)

f=lambda x: x if x<100 else round(random.randint(24,44))
users['Age']=users['Age'].apply(f)
users.describe()

In [ ]:
users['Age'].value_counts()

In [ ]:
users.describe(include=[object])

In [ ]:
users.drop(['city','state','country'],axis=1,inplace=True, errors='ignore')

### Ratings

In [ ]:
ratings.head()

## Content-Based Filtering

## Collaborative Filtering-Matrix Factorization

## Collaborative Filtering-autoencoder Deep Neural Network

## Recommendation System

## Performance Evaluation

## Conclusion